# Experimentations with Entity Embedding

Experimenting with techniques as per the paper:
https://arxiv.org/pdf/1604.06737.pdf

In [1]:
import pandas as pd

In [17]:
train = pd.read_csv('data/train.csv', dtype={'Store': 'object', 'DayOfWeek': 'object',
                                            'Sales': 'int', 'Customers': 'int'}, parse_dates=[2])

/home/brian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [20]:
# buid date features
train['Day'] = train.Date.dt.day
train['Month'] = train.Date.dt.month
train['Year'] = train.Date.dt.year
train['YearWeek'] = train.Date.dt.weekofyear

In [31]:
# load the mapping table
store_map = pd.read_csv('data/store.csv', dtype={'Store': 'object', 'CompetitionOpenSinceMonth': 'object',
                                                 'CompetitionOpenSinceYear': 'object'})
store_map.head(10)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9,2008,0,NaN,NaN,NaN
1,2,a,a,570.0,11,2007,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12,2006,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9,2009,0,NaN,NaN,NaN
4,5,a,a,29910.0,4,2015,0,NaN,NaN,NaN
5,6,a,a,310.0,12,2013,0,NaN,NaN,NaN
6,7,a,c,24000.0,4,2013,0,NaN,NaN,NaN
7,8,a,a,7520.0,10,2014,0,NaN,NaN,NaN
8,9,a,c,2030.0,8,2000,0,NaN,NaN,NaN
9,10,a,a,3160.0,9,2009,0,NaN,NaN,NaN


In [32]:
join_train = train.merge(store_map, left_on='Store', right_on='Store')

In [36]:
join_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Day,...,YearWeek,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,31,...,31,c,a,1270.0,9,2008,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,30,...,31,c,a,1270.0,9,2008,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,29,...,31,c,a,1270.0,9,2008,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,28,...,31,c,a,1270.0,9,2008,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,27,...,31,c,a,1270.0,9,2008,0,NaN,NaN,NaN


In [40]:
Store_feats = join_train[['Store', 'StoreType', 'Assortment']]

In [52]:
from keras.models import Model
from keras.layers import Embedding, Input, Reshape

In [50]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [48]:
Enc_Frame = LabelEncoder()
num_array = Enc_Frame.fit_transform(Store_feats['Store'])

In [51]:
np.unique(num_array)

array([   0,    1,    2, ..., 1112, 1113, 1114])

In [65]:
num_array.shape

(1017209,)

In [97]:
X = np.array(Store_feats)
X[..., [1]]-1

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [82]:
model_input = Input(shape = (1,))
embed_1 = Embedding(input_dim = 1115, 
                    output_dim = 50, 
                    input_length = 100)(model_input)
reshap_1 = Reshape((50,))(embed_1)
model = Model(inputs = model_input, outputs=reshap_1)
model.compile(loss='mean_absolute_error', optimizer='adam')

In [79]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 1, 50)             55750     
_________________________________________________________________
reshape_5 (Reshape)          (None, 50)                0         
Total params: 55,750
Trainable params: 55,750
Non-trainable params: 0
_________________________________________________________________
None


In [90]:
num_array.flatten().shape

(1017209,)

In [91]:
model.fit(x = num_array.flatten(), y = join_train.Sales, batch_size = 1000)

ValueError: Error when checking target: expected reshape_6 to have shape (None, 50) but got array with shape (1017209, 1)